In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
import random
words = open('Indian_Names.txt', 'r').read().splitlines()

random.shuffle(words)
words = [w.lower() for w in words]
words[:8]

['pratul',
 'thanaraj',
 'mantik',
 'yadita',
 'medhani',
 'prathoshan',
 'sumanan',
 'siri']

In [3]:

len(words)

53982

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
print(len(itos))

27


In [6]:

# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

pratul
... ---> p
..p ---> r
.pr ---> a
pra ---> t
rat ---> u
atu ---> l
tul ---> .
thanaraj
... ---> t
..t ---> h
.th ---> a
tha ---> n
han ---> a
ana ---> r
nar ---> a
ara ---> j
raj ---> .
mantik
... ---> m
..m ---> a
.ma ---> n
man ---> t
ant ---> i
nti ---> k
tik ---> .
yadita
... ---> y
..y ---> a
.ya ---> d
yad ---> i
adi ---> t
dit ---> a
ita ---> .
medhani
... ---> m
..m ---> e
.me ---> d
med ---> h
edh ---> a
dha ---> n
han ---> i
ani ---> .


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([38, 3]), torch.int64, torch.Size([38]), torch.int64)

In [8]:
C =  torch.randn((27,2))

In [9]:
emb = C[X]
emb.shape

torch.Size([38, 3, 2])

In [10]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [11]:
h = torch.tanh(emb.view(-1,6) @ W1+b1)

In [12]:
h

tensor([[ 0.2450, -0.9962,  0.6688,  ..., -0.6012,  0.9286, -0.9471],
        [-0.5319, -0.9793,  0.8981,  ..., -0.6327,  0.9904, -0.9098],
        [ 0.6947, -0.9446,  0.3798,  ..., -0.9055,  0.9893, -0.5940],
        ...,
        [-0.9879, -0.9793,  0.4304,  ...,  0.9782, -0.0486, -0.9716],
        [ 0.2279,  0.5933,  0.4355,  ..., -0.4843,  0.9892,  0.9284],
        [-0.6501,  0.0256, -0.9996,  ..., -0.9628, -0.3969, -0.9827]])

In [13]:
h.shape

torch.Size([38, 100])

In [14]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [15]:
logits = h @ W2 + b2
logits.shape

torch.Size([38, 27])

In [16]:
counts = logits.exp()

In [17]:
prob = counts/counts.sum(1,keepdims=True)

In [18]:
prob.shape

torch.Size([38, 27])

In [19]:
Y

tensor([16, 18,  1, 20, 21, 12,  0, 20,  8,  1, 14,  1, 18,  1, 10,  0, 13,  1,
        14, 20,  9, 11,  0, 25,  1,  4,  9, 20,  1,  0, 13,  5,  4,  8,  1, 14,
         9,  0])

In [20]:
loss = -prob[torch.arange(len(Y)), Y].log().mean()
loss

tensor(18.1750)